# Spell-check

This notebook is based on Alura's course [Corretor Ortográfico em Python: aplicando técnicas de NLP (Spell-check: applying NLP techniques)](https://cursos.alura.com.br/course/nlp-corretor-ortografico). Here you will find exploratory data analysis and its conclusions and a spell-check model capable to correct four major mistype problems.

All the code was written in Python 3, using Jupyter Notebooks.

**Check the README file for more information**

--- 

In [1]:
# Creating useful functions

def add_missing_letter(slices):
    """Add missing letter to the mistyped term

    Args:
        slices (list): list with left and right word's slices

    Returns:
        list: list containing all the words' combinations by adding a          
        letter    
    """
    new_words = []
    pt_letters = 'abcdefghijklmnopqrstuvwxyzáâãéêẽíóôõúûç'
    for left_slice, right_slice in slices:
        for letter in pt_letters:
            new_words.append(left_slice + letter + right_slice)
    return new_words


def drop_letter(slices):
    """Drop extra letter of the mistype term

    Args:
        slices (list): list with left and right word's slices

    Returns:
        list: list containing all the words' combinations by dropping a          
        letter
    """
    new_words = []
    for left_slice, right_slice in slices:
        new_words.append(left_slice + right_slice[1:])
    return new_words


def invert_letters(slices):
    """Invert two consecutive letters

    Args:
        slices (list): list with left and right word's slices

    Returns:
        list: list containing all the words' combinations by invert
        letters
    """
    new_words = []
    for left_slice, right_slice in slices:
        if len(right_slice) > 1:
            new_words.append(left_slice +
                             right_slice[1] +
                             right_slice[0] +
                             right_slice[2:])
    return new_words


def lowercase(words_list):
    """Lower case all corpus' letters

    Args:
        words_list (list): list with wordz to lowercase

    Returns:
        list: list with all words in lowercase
    """
    lowercase_list = []
    for palavra in words_list:
        lowercase_list.append(palavra.lower())
    return lowercase_list


def probability(word):
    """Calculates the probability of a term to be the correct one.

    Args:
        word (str): word to be corrected
        articles (str):  

    Returns:
        float: probability of word to be the correct one.
    """
    all_words = len(articles)
    return word_frequency[word]/all_words


def replace_letter(slices):
    """Replace letter of a word to another

    Args:
        slices (list): list with left and right word's slices

    Returns:
        list: list containing all the words' combinations by replacing         
        letters
    """
    new_words = []
    pt_letters = 'abcdefghijklmnopqrstuvwxyzáâãéêẽíóôõúûç'
    for left_slice, right_slice in slices:
        for letter in pt_letters:
            new_words.append(left_slice + letter + right_slice[1:])
    return new_words


def spell_check(word):
    """Correct the mistyped word

    Returns:
        str: correct word
    """
    generated_words = word_generator(word)
    correct_word = max(generated_words, key=probability)
    return correct_word


def spell_check_evaluator(test_set):
    """Evaluates spell-check model performance

    Args:
        test_set (list): list of tuples containing the correct word and
        a mistyped one
    """
    total_words = len(test_set)
    right_words = 0
    for correct, mistyped in test_set:
        correct_word = spell_check(mistyped)
        if correct_word == correct:
            right_words += 1
    right_words_rate = round(right_words*100/total_words, 2)
    print(f"The spell-check success is {right_words_rate}%.")


def split_words(tokens_list):
    """Split tokens to form the corpus
    
    Args:
        tokens_list (list): list with tokens to split

    Returns:
        list: list with all splitted words
    """
    word_list = []
    for token in tokens_list:
        if token.isalpha():
            word_list.append(token)
    return word_list


def word_generator(word):
    """Generates terms to correct the mistyped word

    Args:
        word (str): word to create variations to correct it

    Returns:
        list: list with all generated words (correct and incorrect ones)
    """
    slices = []
    for i in range(len(word)+1):
        slices.append((word[:i],word[i:]))
    formed_words_missing_letter = add_missing_letter(slices)
    formed_words_drop_letter = drop_letter(slices)
    formed_words_replace_letter = replace_letter(slices)
    formed_words_invert_letters = invert_letters(slices)
    formed_words = list(itertools.chain(formed_words_missing_letter,
                                        formed_words_drop_letter,
                                        formed_words_replace_letter,
                                        formed_words_invert_letters))
    return formed_words


def upload_test_set(file_name):
    """Uploads spell-check test set

    Args:
        file_name (str): file name containing the test set

    Returns:
        list: list of tuples with the correct term and its mistyped version
    """
    test_set = []
    f = open(file_name, "r", encoding="utf8")
    for line in f:
        corret, mistyped = line.split()
        test_set.append((corret, mistyped))
    f.close()
    return test_set


# EDA

In [2]:
# Importing libraries

import itertools
import nltk

In [3]:
# Opening txt file

with open("artigos.txt", "r",encoding="utf8") as f:
    articles = f.read()

In [4]:
# Checking the number of characters in the corpus

len(articles)

2605046

Let's count the most common words in the corpus.

In [5]:
# Counting the most common words

tokens_frequency = nltk.tokenize.word_tokenize(articles)
word_corpus = lowercase(split_words(tokens_frequency))
word_frequency = nltk.FreqDist(word_corpus)
word_frequency.most_common(10)

[('de', 15494),
 ('o', 13969),
 ('que', 12225),
 ('a', 11041),
 ('e', 10480),
 ('para', 7695),
 ('um', 6346),
 ('é', 5881),
 ('uma', 5202),
 ('do', 5118)]

There are several words with a frequency higher than 5000 and most of them are stop words. But to build spell check, it is necessary a large number of unique terms. Let's count them.

In [6]:
# Counting unique words in the corpus

unique_words_corpus = set(word_corpus)
len(unique_words_corpus)

18057

The corpus has about 18000 unique terms. According to this [article](https://www.optilingo.com/blog/general/how-many-words-do-you-need-to-know-to-become-fluent-in-a-language/#:~:text=Knowing%204%2C000%20to%2010%2C000%20words,fluent%20or%20native%2Dspeaker%20levels.) it is necessary 4000 to 10000 words to be considered advanced level. Therefore, we have enough words to build a good first version of the spell check.

---

# Missing letter

There are several problems spell check can solve. The first one is the missing letter, i.e. when one letter is not present in the typed word. To figure out this problem, we can slice the word into left and right pieces and add one letter at a time to form a word, and then calculate the probability that the word is a valid one. For example, we can misspell "mchine" and the algorithm will slice the term into "m" (left) and "chine" (right) and add the letters a,b,c to form "machine", "mbchine", "mcchine" respectively and, then pick the first term since the last two are not actual words using probability.

Let's test the spell check using a Portuguese word for logic: "lógica".

In [7]:
# Creating a misspelled word

test_word = "lgica"

In [8]:
# Generating new words

generated_words = word_generator(test_word)
print(generated_words)

['algica', 'blgica', 'clgica', 'dlgica', 'elgica', 'flgica', 'glgica', 'hlgica', 'ilgica', 'jlgica', 'klgica', 'llgica', 'mlgica', 'nlgica', 'olgica', 'plgica', 'qlgica', 'rlgica', 'slgica', 'tlgica', 'ulgica', 'vlgica', 'wlgica', 'xlgica', 'ylgica', 'zlgica', 'álgica', 'âlgica', 'ãlgica', 'élgica', 'êlgica', 'ẽlgica', 'ílgica', 'ólgica', 'ôlgica', 'õlgica', 'úlgica', 'ûlgica', 'çlgica', 'lagica', 'lbgica', 'lcgica', 'ldgica', 'legica', 'lfgica', 'lggica', 'lhgica', 'ligica', 'ljgica', 'lkgica', 'llgica', 'lmgica', 'lngica', 'logica', 'lpgica', 'lqgica', 'lrgica', 'lsgica', 'ltgica', 'lugica', 'lvgica', 'lwgica', 'lxgica', 'lygica', 'lzgica', 'lágica', 'lâgica', 'lãgica', 'légica', 'lêgica', 'lẽgica', 'lígica', 'lógica', 'lôgica', 'lõgica', 'lúgica', 'lûgica', 'lçgica', 'lgaica', 'lgbica', 'lgcica', 'lgdica', 'lgeica', 'lgfica', 'lggica', 'lghica', 'lgiica', 'lgjica', 'lgkica', 'lglica', 'lgmica', 'lgnica', 'lgoica', 'lgpica', 'lgqica', 'lgrica', 'lgsica', 'lgtica', 'lguica', 'lgvica',

Here it is possible to see all the words that were created and the correct one is among them. To pick it, the algorithm will use probability. Let's look at examples to see the probability for logica and lógica be the right word.

In [9]:
# Printing the probabilities for the right and wrong terms being the correct one

print("The probability of 'lógica' being the right word is", probability("lógica")*100)
print("The probability of 'logica' being the right word is", probability("logica")*100)

The probability of 'lógica' being the right word is 0.003339672312888141
The probability of 'logica' being the right word is 0.0


The correct word has a probability slightly above 0 and the wrong one is exactly 0.0. Therefore, the algorithm is capable to select the right word. 

Finally, let's test the spell check for missing letter problems.

In [10]:
# Testing spell check for missing letters

spell_check(test_word)

'lógica'

In [11]:
generated_words = word_generator(test_word)

In [12]:
correct_word = max(generated_words, key=probability)

Voilá! The spell check is working properly. But the missing letter is not the only problem when we mistype a word. Let's improve the model to cover all the possible issues.

--- 

# Drop letter

Another common mistype problem happens when we add an extra letter that does not belong to the correct word. For example, the word "machine" could have been incorrectly typed as machione. The approach to solve this problem is similar to the previous model: instead of slicing words in different formats to add a new letter, the slice on the right side will lose its first letter and then concatenate to form a new word (correct or not). Finally, the model will use probability to pick the correct term. 

To implement it, it is necessary to adapt the word_generator function to concatenate the list of words returned by the drop_letter function. Let's check the terms created by dropping letter from mistyped word loigica.

In [13]:
# Creating terms to correct mistyped word

test_word_drop_letter = "lóigica" 
word_generator(test_word_drop_letter)

['alóigica',
 'blóigica',
 'clóigica',
 'dlóigica',
 'elóigica',
 'flóigica',
 'glóigica',
 'hlóigica',
 'ilóigica',
 'jlóigica',
 'klóigica',
 'llóigica',
 'mlóigica',
 'nlóigica',
 'olóigica',
 'plóigica',
 'qlóigica',
 'rlóigica',
 'slóigica',
 'tlóigica',
 'ulóigica',
 'vlóigica',
 'wlóigica',
 'xlóigica',
 'ylóigica',
 'zlóigica',
 'álóigica',
 'âlóigica',
 'ãlóigica',
 'élóigica',
 'êlóigica',
 'ẽlóigica',
 'ílóigica',
 'ólóigica',
 'ôlóigica',
 'õlóigica',
 'úlóigica',
 'ûlóigica',
 'çlóigica',
 'laóigica',
 'lbóigica',
 'lcóigica',
 'ldóigica',
 'leóigica',
 'lfóigica',
 'lgóigica',
 'lhóigica',
 'lióigica',
 'ljóigica',
 'lkóigica',
 'llóigica',
 'lmóigica',
 'lnóigica',
 'loóigica',
 'lpóigica',
 'lqóigica',
 'lróigica',
 'lsóigica',
 'ltóigica',
 'luóigica',
 'lvóigica',
 'lwóigica',
 'lxóigica',
 'lyóigica',
 'lzóigica',
 'láóigica',
 'lâóigica',
 'lãóigica',
 'léóigica',
 'lêóigica',
 'lẽóigica',
 'líóigica',
 'lóóigica',
 'lôóigica',
 'lõóigica',
 'lúóigica',
 'lûóigica',

Let's test if the model can correct the mistyped word lóigica.

In [14]:
# Testing spell check for drop letter


spell_check(test_word_drop_letter)

'lógica'

All right! It seems that the spell-check is also working with extra letter words. But the algorithm can handle more complex tasks.

--- 

# Replaced letter

One more common problem is when one accidentally replaced a letter with another to form an incorrect term. For example, the word machine could be typed as machone. The length and the general word format are correct except for the letter "o" instead of "i". To solve this problem the algorithm needs to slice the wrong term into right and left pieces and drop the first letter of the latter one and add new letters in that position, and then it concatenates altogether. 

We need to change again the function word_generator to include a new one called replace_letter and concatenate with other lists into a single variable containing all the terms created to correct the mistyped word.

Let's test the third version of the spell-check with the term lígica.

In [15]:
# Creating terms to correct mistyped word

test_word_replace_letter = "lígica" 
word_generator(test_word_replace_letter)

['alígica',
 'blígica',
 'clígica',
 'dlígica',
 'elígica',
 'flígica',
 'glígica',
 'hlígica',
 'ilígica',
 'jlígica',
 'klígica',
 'llígica',
 'mlígica',
 'nlígica',
 'olígica',
 'plígica',
 'qlígica',
 'rlígica',
 'slígica',
 'tlígica',
 'ulígica',
 'vlígica',
 'wlígica',
 'xlígica',
 'ylígica',
 'zlígica',
 'álígica',
 'âlígica',
 'ãlígica',
 'élígica',
 'êlígica',
 'ẽlígica',
 'ílígica',
 'ólígica',
 'ôlígica',
 'õlígica',
 'úlígica',
 'ûlígica',
 'çlígica',
 'laígica',
 'lbígica',
 'lcígica',
 'ldígica',
 'leígica',
 'lfígica',
 'lgígica',
 'lhígica',
 'liígica',
 'ljígica',
 'lkígica',
 'llígica',
 'lmígica',
 'lnígica',
 'loígica',
 'lpígica',
 'lqígica',
 'lrígica',
 'lsígica',
 'ltígica',
 'luígica',
 'lvígica',
 'lwígica',
 'lxígica',
 'lyígica',
 'lzígica',
 'láígica',
 'lâígica',
 'lãígica',
 'léígica',
 'lêígica',
 'lẽígica',
 'líígica',
 'lóígica',
 'lôígica',
 'lõígica',
 'lúígica',
 'lûígica',
 'lçígica',
 'líagica',
 'líbgica',
 'lícgica',
 'lídgica',
 'líegica',
 'lí

In [16]:
# Testing spell check for replaced letters

spell_check(test_word_replace_letter)

'lógica'

One more problem the spell-check can solve from now on. But we can implement one final problem for this algorithm to solve.

--- 

# Invert letters

A similar problem to replacing letters is when someone inverts a letter accidentally. For example, instead of the word machine, the mistyped term is macihne. In this case, it is necessary to swap positions between the letters "i" and "h". To implement this solution, it is required to slice the wrong term into right and left sides and take the latter and invert the first two letters with each other and then concatenate altogether. 

We need to create another function to invert letters position and then include it into the word_generator function. After that, we can test the model using the mistyped word lgóica.

In [17]:
# Creating terms to correct mistyped word

test_word_inverted_letters = "lgóica" 
word_generator(test_word_inverted_letters)

['algóica',
 'blgóica',
 'clgóica',
 'dlgóica',
 'elgóica',
 'flgóica',
 'glgóica',
 'hlgóica',
 'ilgóica',
 'jlgóica',
 'klgóica',
 'llgóica',
 'mlgóica',
 'nlgóica',
 'olgóica',
 'plgóica',
 'qlgóica',
 'rlgóica',
 'slgóica',
 'tlgóica',
 'ulgóica',
 'vlgóica',
 'wlgóica',
 'xlgóica',
 'ylgóica',
 'zlgóica',
 'álgóica',
 'âlgóica',
 'ãlgóica',
 'élgóica',
 'êlgóica',
 'ẽlgóica',
 'ílgóica',
 'ólgóica',
 'ôlgóica',
 'õlgóica',
 'úlgóica',
 'ûlgóica',
 'çlgóica',
 'lagóica',
 'lbgóica',
 'lcgóica',
 'ldgóica',
 'legóica',
 'lfgóica',
 'lggóica',
 'lhgóica',
 'ligóica',
 'ljgóica',
 'lkgóica',
 'llgóica',
 'lmgóica',
 'lngóica',
 'logóica',
 'lpgóica',
 'lqgóica',
 'lrgóica',
 'lsgóica',
 'ltgóica',
 'lugóica',
 'lvgóica',
 'lwgóica',
 'lxgóica',
 'lygóica',
 'lzgóica',
 'lágóica',
 'lâgóica',
 'lãgóica',
 'légóica',
 'lêgóica',
 'lẽgóica',
 'lígóica',
 'lógóica',
 'lôgóica',
 'lõgóica',
 'lúgóica',
 'lûgóica',
 'lçgóica',
 'lgaóica',
 'lgbóica',
 'lgcóica',
 'lgdóica',
 'lgeóica',
 'lg

In [18]:
# Testing spell check for inverted letters

spell_check(test_word_inverted_letters)

'lógica'

One more time, the spell-check can correctly return the right term for a mistyped word.

---

# Model evaluation

It is necessary to evaluate if the spell check is working or not. And this process is similar to the most common machine learning algorithms. There is a test set containing correct words and their mistyped versions and the goal is to correct the highest number of terms.

Let's import the test set to evaluate the spell-check first model.

In [19]:
# Importing the test set

test_set = upload_test_set("palavras.txt")
test_set

[('podemos', 'pyodemos'),
 ('esse', 'esje'),
 ('já', 'jrá'),
 ('nosso', 'nossov'),
 ('são', 'sãêo'),
 ('dos', 'dosa'),
 ('muito', 'muifo'),
 ('imagem', 'iômagem'),
 ('sua', 'ósua'),
 ('também', 'tambéùm'),
 ('ele', 'eme'),
 ('fazer', 'èazer'),
 ('temos', 'temfs'),
 ('essa', 'eàssa'),
 ('quando', 'quaôdo'),
 ('vamos', 'vamvos'),
 ('sobre', 'hsobre'),
 ('java', 'sjava'),
 ('das', 'daõs'),
 ('agora', 'agorah'),
 ('está', 'eòtá'),
 ('cada', 'céda'),
 ('mesmo', 'zmesmo'),
 ('nos', 'noâ'),
 ('forma', 'fobma'),
 ('seja', 'sejéa'),
 ('então', 'enêão'),
 ('criar', 'èriar'),
 ('código', 'cóeigo'),
 ('caso', 'casío'),
 ('exemplo', 'áexemplo'),
 ('tem', 'tĩem'),
 ('usuário', 'usuárôio'),
 ('dados', 'dfados'),
 ('python', 'pgthon'),
 ('nossa', 'nossah'),
 ('além', 'alémè'),
 ('assim', 'asõim'),
 ('ter', 'teb'),
 ('até', 'atĩ'),
 ('bem', 'âem'),
 ('design', 'desigen'),
 ('trabalho', 'trabalàho'),
 ('foi', 'foo'),
 ('apenas', 'apenaũ'),
 ('empresa', 'empresà'),
 ('valor', 'valíor'),
 ('será', 'serr')

In [20]:
# Evaluating first spell-check model

spell_check_evaluator(test_set)

The spell-check success is 76.34%.


The correction success is about 76,34% covering the most common mistype problems. Let's remember that the model is based on those around 18000 words in the corpus. We need to check if all these test words are in it because the missing 24% could be linked to corpus size limitations. 

In [21]:
# Counting how many test words are not in the corpus

total_missing_words = 0
corpus_missing_words = []
for correct_word, _ in test_set:
    if correct_word not in unique_words_corpus:
        total_missing_words += 1
        corpus_missing_words.append(correct_word)

percentual_missing_words = round(total_missing_words/len(test_set)*100, 2)
print(f"The number of total test words that are not in the corpus is {total_missing_words}, which is {percentual_missing_words}% of all terms in the test set.")


The number of total test words that are not in the corpus is 13, which is 6.99% of all terms in the test set.


In [22]:
corpus_missing_words

['empoderamento',
 'cavalo',
 'canela',
 'tênis',
 'ansiosa',
 'ansiosa',
 'ansiosa',
 'empoderamento',
 'entretido',
 'idiota',
 'prevalecer',
 'mendigo',
 'perturbar']

There are two repeated words into the test set that are missing in the corpus. Let's include the 11 words from the test set into the corpus and calculate again the model's performance.

In [23]:
# Adding the missing words to the corpus

for word in corpus_missing_words:
    unique_words_corpus.add(word)

In [24]:
len(unique_words_corpus)

18067

Let's test again the model's performance using these new 11 words in the corpus.

In [25]:
# Evaluating the final spell-check model

spell_check_evaluator(test_set)

The spell-check success is 76.34%.


Adding these missing words did not make difference, since 11 is only 0.06% of the corpus.

--- 

# Recommendations

To achieve nearly 100% of the right corrections, we could expand the corpus to include more words and search them from different contexts and sources to add a big variety of daily basis words, technical terms, and verb variations, for example.